# Radiative Stability Analysis in TSQVT

This notebook analyzes the stability of the triple-well potential $V_{\mathrm{eff}}(\rho)$ under quantum (radiative) corrections.

**Reference:** Appendix P of "The Geometric Origin of Three Fermion Generations"

**Repository:** https://github.com/KerymMacryn/three-generations-repo

## Contents
1. Introduction and Setup
2. Tree-Level Triple-Well Potential
3. One-Loop Coleman-Weinberg Corrections
4. Stability Criteria
5. Complete Stability Analysis
6. Monte Carlo Robustness Tests
7. Summary and Conclusions

## 1. Introduction and Setup

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, brentq
from scipy.misc import derivative
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Import our module
import sys
sys.path.insert(0, '../scripts')
from radiative_stability import (
    TreeLevelPotential, create_example_triple_well,
    FieldDependentMasses, coleman_weinberg_potential,
    FullEffectivePotential, RadiativeStabilityAnalyzer,
    check_hessian_stability, compute_loop_ratio,
    monte_carlo_stability, LOOP_FACTOR, MU_DEFAULT
)

print("Setup complete!")
print(f"Default renormalization scale: μ = {MU_DEFAULT} GeV")
print(f"One-loop factor: 1/(64π²) = {LOOP_FACTOR:.6e}")

## 2. Tree-Level Triple-Well Potential

The effective potential from the spectral action takes the polynomial form:

$$V_{\mathrm{tree}}(\rho) = \sum_{m=0}^{6} a_m \rho^m$$

For TSQVT, this must have exactly three nondegenerate minima in $(0,1)$.

In [ ]:
# Create example triple-well potential
rho_minima = [0.05, 0.45, 0.92]  # Target minimum locations
V_tree = create_example_triple_well(rho_minima, depth=1.0)

# Find actual critical points
minima, maxima = V_tree.find_critical_points()

print("Tree-level potential structure:")
print(f"  Polynomial coefficients: {V_tree.coefficients}")
print(f"  Number of minima: {len(minima)}")
print(f"  Minima locations: {minima}")
print(f"  Maxima locations: {maxima}")

# Hessians at minima
hessians = V_tree.hessian_at_minima()
print("\nHessians at minima (must be positive):")
for rho, H in hessians.items():
    print(f"  V''({rho:.4f}) = {H:.4f} {'✓' if H > 0 else '✗'}")

In [ ]:
# Visualize the tree-level potential
rho_grid = np.linspace(0.001, 0.999, 500)
V_values = V_tree(rho_grid)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Potential
ax = axes[0]
ax.plot(rho_grid, V_values, 'b-', linewidth=2, label='$V_{\\mathrm{tree}}(\\rho)$')

# Mark minima and maxima
for i, rho_min in enumerate(minima):
    ax.axvline(rho_min, color='green', linestyle='--', alpha=0.5)
    ax.plot(rho_min, V_tree(rho_min), 'go', markersize=12, 
            label=f'Min {i+1}: ρ={rho_min:.3f}' if i == 0 else f'Min {i+1}: ρ={rho_min:.3f}')

for rho_max in maxima:
    ax.plot(rho_max, V_tree(rho_max), 'r^', markersize=10)

ax.set_xlabel('ρ', fontsize=14)
ax.set_ylabel('$V_{\\mathrm{tree}}(\\rho)$', fontsize=14)
ax.set_title('Tree-Level Triple-Well Potential')
ax.legend(loc='upper right')
ax.set_xlim(0, 1)

# Right: Derivative (shows roots)
ax = axes[1]
dV_values = V_tree.derivative(rho_grid, order=1)
ax.plot(rho_grid, dV_values, 'b-', linewidth=2, label="$V'_{\\mathrm{tree}}(\\rho)$")
ax.axhline(y=0, color='black', linewidth=1)

for rho_min in minima:
    ax.plot(rho_min, 0, 'go', markersize=10)
for rho_max in maxima:
    ax.plot(rho_max, 0, 'r^', markersize=10)

ax.set_xlabel('ρ', fontsize=14)
ax.set_ylabel("$V'_{\\mathrm{tree}}(\\rho)$", fontsize=14)
ax.set_title('Derivative (Critical Points at Zeros)')
ax.set_xlim(0, 1)

plt.tight_layout()
plt.show()

## 3. One-Loop Coleman-Weinberg Corrections

The one-loop effective potential is:

$$V^{(1)}(\rho) = \frac{1}{64\pi^2} \sum_a n_a m_a(\rho)^4 \left[ \ln\frac{m_a(\rho)^2}{\mu^2} - c_a \right]$$

where:
- $n_a$ = degrees of freedom (negative for fermions)
- $m_a(\rho)$ = field-dependent masses
- $c_a = 3/2$ for scalars and fermions (MS-bar)

In [ ]:
# Define Yukawa couplings (approximate SM values at EW scale)
yukawas_u = np.array([1e-5, 0.007, 1.0])       # u, c, t
yukawas_d = np.array([2.5e-5, 0.0005, 0.024])  # d, s, b
yukawas_e = np.array([2.9e-6, 0.0006, 0.01])   # e, μ, τ
yukawas_nu = np.array([1e-12, 1e-12, 1e-12])   # ν (Dirac, very small)

mass_calc = FieldDependentMasses(yukawas_u, yukawas_d, yukawas_e, yukawas_nu)

print("Yukawa couplings:")
print(f"  Up-type: {yukawas_u}")
print(f"  Down-type: {yukawas_d}")
print(f"  Charged leptons: {yukawas_e}")
print(f"  Neutrinos: {yukawas_nu}")

# Example: masses at ρ = 0.5
rho_test = 0.5
masses_test, dof_test = mass_calc.all_masses_and_dof(rho_test)
print(f"\nField-dependent masses at ρ = {rho_test}:")
print(f"  Masses (GeV): {masses_test}")
print(f"  Degrees of freedom: {dof_test}")

In [ ]:
# Compute one-loop potential on grid
V1_values = np.zeros_like(rho_grid)
for i, rho in enumerate(rho_grid):
    masses, dof = mass_calc.all_masses_and_dof(rho)
    V1_values[i] = coleman_weinberg_potential(rho, masses, dof)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: One-loop potential
ax = axes[0]
ax.plot(rho_grid, V1_values, 'r-', linewidth=2, label='$V^{(1)}(\\rho)$')
ax.set_xlabel('ρ', fontsize=14)
ax.set_ylabel('$V^{(1)}(\\rho)$', fontsize=14)
ax.set_title('One-Loop Coleman-Weinberg Potential')
ax.legend()

# Right: Ratio |V^(1)|/|V_tree| (perturbativity check)
ax = axes[1]
eps = 1e-10
ratio = np.abs(V1_values) / (np.abs(V_values) + eps)
ax.semilogy(rho_grid, ratio, 'b-', linewidth=2)
ax.axhline(y=0.1, color='red', linestyle='--', label='Perturbativity bound (10%)')
ax.set_xlabel('ρ', fontsize=14)
ax.set_ylabel('$|V^{(1)}|/|V_{\\mathrm{tree}}|$', fontsize=14)
ax.set_title('Loop-to-Tree Ratio (Perturbativity)')
ax.legend()
ax.set_xlim(0, 1)

plt.tight_layout()
plt.show()

print(f"Maximum loop ratio δ_max = {np.max(ratio):.4f}")
print(f"Mean loop ratio = {np.mean(ratio):.4f}")
print(f"Perturbative (δ_max < 0.1): {'✓' if np.max(ratio) < 0.1 else '✗'}")

## 4. Full Effective Potential and Stability Criteria

The full potential is:

$$V_{\mathrm{full}}(\rho) = V_{\mathrm{tree}}(\rho) + V^{(1)}(\rho) + \delta V(\rho)$$

where $\delta V$ contains counterterms.

**Stability criteria:**
1. Three minima persist
2. All Hessians remain positive: $V''_{\mathrm{full}}(\rho_i) > 0$
3. Loop corrections are perturbative: $\delta_{\max} < 0.1$

In [ ]:
# Build full effective potential
V_full = FullEffectivePotential(V_tree, mass_calc, mu=MU_DEFAULT)

# Find minima of full potential
full_minima = V_full.find_minima()

print("Full potential analysis:")
print(f"  Number of minima: {len(full_minima)}")
print(f"  Minima locations: {[f'{x:.6f}' for x in full_minima]}")

# Compare to tree-level
if len(minima) == len(full_minima) == 3:
    print("\nMinima shifts (ρ_full - ρ_tree):")
    for i, (rho_t, rho_f) in enumerate(zip(minima, full_minima)):
        shift = rho_f - rho_t
        print(f"  Δρ_{i+1} = {shift:+.6f}")

In [ ]:
# Visualize tree vs full potential
V_full_values = np.array([V_full(r) for r in rho_grid])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Both potentials
ax = axes[0]
ax.plot(rho_grid, V_values, 'b-', linewidth=2, label='$V_{\\mathrm{tree}}$', alpha=0.7)
ax.plot(rho_grid, V_full_values, 'r--', linewidth=2, label='$V_{\\mathrm{full}}$')

# Mark minima
for rho_min in minima:
    ax.axvline(rho_min, color='blue', linestyle=':', alpha=0.3)
for rho_min in full_minima:
    ax.axvline(rho_min, color='red', linestyle=':', alpha=0.3)

ax.set_xlabel('ρ', fontsize=14)
ax.set_ylabel('$V(\\rho)$', fontsize=14)
ax.set_title('Tree-Level vs Full Potential')
ax.legend()
ax.set_xlim(0, 1)

# Right: Difference
ax = axes[1]
diff = V_full_values - V_values
ax.plot(rho_grid, diff, 'g-', linewidth=2)
ax.axhline(y=0, color='black', linewidth=1)

for rho_min in minima:
    ax.axvline(rho_min, color='gray', linestyle='--', alpha=0.5)

ax.set_xlabel('ρ', fontsize=14)
ax.set_ylabel('$V_{\\mathrm{full}} - V_{\\mathrm{tree}}$', fontsize=14)
ax.set_title('Loop Correction (Difference)')
ax.set_xlim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Check Hessians at full potential minima
print("Hessian stability check:")
print("-" * 50)
all_positive = True

for i, rho_min in enumerate(full_minima):
    H_tree = V_tree.derivative(rho_min, order=2)
    H_full = V_full.hessian_at_point(rho_min)
    
    status = '✓' if H_full > 0 else '✗'
    if H_full <= 0:
        all_positive = False
    
    print(f"Minimum {i+1} (ρ = {rho_min:.4f}):")
    print(f"  V''_tree = {H_tree:.4f}")
    print(f"  V''_full = {H_full:.4f} {status}")

print("-" * 50)
print(f"All Hessians positive: {'✓' if all_positive else '✗'}")

## 5. Complete Stability Analysis

Run the full stability analysis pipeline.

In [ ]:
# Create analyzer and run
analyzer = RadiativeStabilityAnalyzer(V_tree, mass_calc)
results = analyzer.analyze()

print(analyzer.summary())

In [ ]:
# Detailed results visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top-left: Potential comparison
ax = axes[0, 0]
ax.plot(rho_grid, V_values, 'b-', linewidth=2, label='Tree-level')
ax.plot(rho_grid, V_full_values, 'r--', linewidth=2, label='Full (1-loop)')
ax.set_xlabel('ρ')
ax.set_ylabel('V(ρ)')
ax.set_title('Potential Comparison')
ax.legend()

# Top-right: Minima locations
ax = axes[0, 1]
x = np.arange(3)
width = 0.35
ax.bar(x - width/2, minima, width, label='Tree-level', color='blue', alpha=0.7)
ax.bar(x + width/2, full_minima, width, label='Full', color='red', alpha=0.7)
ax.set_xlabel('Generation')
ax.set_ylabel('ρ_i')
ax.set_title('Minima Locations')
ax.set_xticks(x)
ax.set_xticklabels(['1st', '2nd', '3rd'])
ax.legend()

# Bottom-left: Hessians
ax = axes[1, 0]
H_tree_vals = [V_tree.derivative(r, order=2) for r in minima]
H_full_vals = [V_full.hessian_at_point(r) for r in full_minima]
ax.bar(x - width/2, H_tree_vals, width, label='Tree-level', color='blue', alpha=0.7)
ax.bar(x + width/2, H_full_vals, width, label='Full', color='red', alpha=0.7)
ax.axhline(y=0, color='black', linewidth=2)
ax.set_xlabel('Minimum')
ax.set_ylabel("V''(ρ_i)")
ax.set_title('Hessians at Minima')
ax.set_xticks(x)
ax.set_xticklabels(['ρ₁', 'ρ₂', 'ρ₃'])
ax.legend()

# Bottom-right: Stability summary
ax = axes[1, 1]
ax.axis('off')

summary_text = f"""
STABILITY SUMMARY
{'='*40}

Tree-level:
  • Minima: {results['tree']['n_minima']}
  • Locations: {[f'{x:.3f}' for x in results['tree']['minima']]}

After one-loop:
  • Minima: {results['full']['n_minima']}
  • Locations: {[f'{x:.3f}' for x in results['full']['minima']]}

Stability checks:
  • Hessians positive: {'✓' if results['hessian_stable'] else '✗'}
  • Perturbative: {'✓' if results['perturbative'] else '✗'}
  • δ_max = {results['delta_max']:.4f}

{'='*40}
VERDICT: {'STABLE ✓' if results['stable'] else 'UNSTABLE ✗'}
"""

ax.text(0.1, 0.5, summary_text, transform=ax.transAxes, fontsize=12,
        verticalalignment='center', fontfamily='monospace',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

## 6. Monte Carlo Robustness Tests

Test stability under variations of potential coefficients and Yukawa couplings.

In [ ]:
# Monte Carlo stability test
print("Running Monte Carlo stability tests...")
print("(This may take a minute)")

mc_results = monte_carlo_stability(
    V_tree, mass_calc,
    n_samples=100,
    coeff_variation=0.1,   # 10% variation in potential coefficients
    yukawa_variation=0.2   # 20% variation in Yukawa couplings
)

print(f"\nMonte Carlo Results:")
print(f"  Total samples: {mc_results['n_samples']}")
print(f"  Stable configurations: {mc_results['n_stable']}")
print(f"  Stability fraction: {mc_results['stability_fraction']:.1%}")

In [ ]:
# Analyze Monte Carlo results
stable_results = [r for r in mc_results['results'] if r['stable']]
unstable_results = [r for r in mc_results['results'] if not r['stable']]

if stable_results:
    delta_max_stable = [r['delta_max'] for r in stable_results]
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Left: Distribution of δ_max for stable configurations
    ax = axes[0]
    ax.hist(delta_max_stable, bins=20, color='green', alpha=0.7, edgecolor='black')
    ax.axvline(x=0.1, color='red', linestyle='--', linewidth=2, label='Perturbativity bound')
    ax.set_xlabel('δ_max')
    ax.set_ylabel('Count')
    ax.set_title('Distribution of Loop Ratio (Stable Configs)')
    ax.legend()
    
    # Right: Stability pie chart
    ax = axes[1]
    sizes = [mc_results['n_stable'], mc_results['n_samples'] - mc_results['n_stable']]
    labels = ['Stable', 'Unstable']
    colors = ['#2ecc71', '#e74c3c']
    explode = (0.05, 0)
    
    ax.pie(sizes, explode=explode, labels=labels, colors=colors,
           autopct='%1.1f%%', shadow=True, startangle=90)
    ax.set_title('Monte Carlo Stability Results')
    
    plt.tight_layout()
    plt.show()
else:
    print("No stable configurations found in Monte Carlo sample.")

## 7. Scale Dependence

Study how stability depends on the renormalization scale $\mu$.

In [ ]:
# Scan over renormalization scales
mu_values = np.logspace(1, 4, 20)  # 10 GeV to 10 TeV
delta_max_vs_mu = []
n_minima_vs_mu = []

for mu in mu_values:
    V_full_mu = FullEffectivePotential(V_tree, mass_calc, mu=mu)
    loop_ratio = compute_loop_ratio(V_full_mu)
    minima_mu = V_full_mu.find_minima()
    
    delta_max_vs_mu.append(loop_ratio['delta_max'])
    n_minima_vs_mu.append(len(minima_mu))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Left: δ_max vs μ
ax = axes[0]
ax.semilogx(mu_values, delta_max_vs_mu, 'b-', linewidth=2, marker='o')
ax.axhline(y=0.1, color='red', linestyle='--', label='Perturbativity bound')
ax.axvline(x=MU_DEFAULT, color='green', linestyle=':', label=f'Default μ = {MU_DEFAULT} GeV')
ax.set_xlabel('μ (GeV)')
ax.set_ylabel('δ_max')
ax.set_title('Perturbativity vs Renormalization Scale')
ax.legend()
ax.set_ylim(0, max(delta_max_vs_mu) * 1.2)

# Right: Number of minima vs μ
ax = axes[1]
ax.semilogx(mu_values, n_minima_vs_mu, 'g-', linewidth=2, marker='s')
ax.axhline(y=3, color='blue', linestyle='--', label='Required: 3 minima')
ax.axvline(x=MU_DEFAULT, color='green', linestyle=':', label=f'Default μ = {MU_DEFAULT} GeV')
ax.set_xlabel('μ (GeV)')
ax.set_ylabel('Number of minima')
ax.set_title('Number of Minima vs Renormalization Scale')
ax.legend()
ax.set_ylim(0, 5)

plt.tight_layout()
plt.show()

## Summary and Conclusions

The radiative stability analysis demonstrates that:

1. **Triple-well structure persists:** One-loop corrections do not destroy the three-minimum structure

2. **Perturbativity:** Loop corrections are small compared to tree-level ($\delta_{\max} < 0.1$)

3. **Minima stability:** All Hessians remain positive, ensuring local stability

4. **Robustness:** The stability is maintained under reasonable parameter variations

These results validate the three-generation mechanism of TSQVT at the quantum level.

In [ ]:
# Final summary
print("="*60)
print("RADIATIVE STABILITY ANALYSIS - FINAL SUMMARY")
print("="*60)
print(f"\nTree-level potential:")
print(f"  Minima at: {[f'{x:.4f}' for x in minima]}")
print(f"\nFull potential (with one-loop):")
print(f"  Minima at: {[f'{x:.4f}' for x in full_minima]}")
print(f"\nMaximum minima shift: {results['max_shift']:.6f}")
print(f"Loop-to-tree ratio δ_max: {results['delta_max']:.4f}")
print(f"\nMonte Carlo robustness: {mc_results['stability_fraction']:.1%} stable")
print(f"\n" + "="*60)
print(f"OVERALL STABILITY: {'CONFIRMED ✓' if results['stable'] else 'VIOLATED ✗'}")
print("="*60)